# $$Project Description$$ #

DATA 1030

Kaiwen Yang

## Introduction

This project aims to use machine learning methods to predict the offensive win share of a NBA player base on his shooting tendency. Win share is an advanced player stat that indicates how much a player contributes to success of his team. It is an important attribute to evaluate a player. In this project, two datasets will be used. "Seasons_Stats.csv" contains the basic stats of all NBA players from 1950 to 2017. "nba_shot_types.csv" contains data about players' shooting tendency (e.g. the ratio of a player's shots inside three point line to shots outside the three point line). The target variable would be "ows", which is the offensive win share of a player. It is a continuous variable, so regression model would be implemented. 

The topic of this project is highly related to the ongoing revolution of NBA. A lot of teams intorduced data science to game play analysis in order to improve offense and defense efficiency. They use more complex and detailed dataset to find out the most efficient offense choices, and most of the results suggest that NBA teams should encourage their players to shot more three-point and shot less mid-range jump shots. In this project, we could use simpler dataset to reproduce this analysis. 

## Dataset

In "nba_shot_types.csv", there are 3007 data points, and each has 23 features. Here are column descriptions for columns that will be using: 

"YEAR": NBA season. Category. Will be changed to discrete numerical data for the convinience of merging. 

"PLAYER": Player name. Category. 

The following data type are all float: 

"PCT_FGA_2PT": Percentage of Field Goal Attempts That Were 2 PT Shots

"PCT_FGA_3PT": Percentage of Field Goal Attempts That Were 2 PT Shots

"PCT_PTS_2PT": Percentage of Points That Came From 2 PT Field Goals Made

"PCT_PTS_MR": Percentage of Points That Came From Midrange

"PCT_PTS_3PT": Percentage of Points That Came From 3 PT Field Goals Made

"PCT_PTS_FSTBRK": Percentage of Points That Came on Fast Breaks

"PCT_PTS_FT": Percentage of Points That Came From Free Throws

"PCT_PTS_OFF_TOS": Percentage of Points That Came Off Turnovers

"PCT_PTS_INTHEPT": Percentage of Points That Came In the Paint

In "Seasons_stats_complete.csv", there are 26.1k of data points and each has 50 features. Only data between 2013-2019 will be used. Here are column descriptions for columns that will be using:

"Year": NBA season. Category. Will be changed to discrete numerical data for the convinience of merging. 

"Player": Player name. Category. 

"Pos": Position. Category. 

"OWS": offensive win share. Continuous numerical.

"2PA": 2-point attempt. Discrete numerical.

"3PA": 3-point attempt. Discrete numerical.

"FTA": free throw attempt. Discrete numerical.

## Preprocessing of Datasets

Two tables will be merged base on the "Year" and "Player" columns. After merging, these two columns will be droped.

one-hot encoder will be applied to: "Pos". The 5 different position are not ordinal. They just stand for the position of the player plays in that season. 

MinMaxEncoder will be applied to columns "2PA", "3PA", "FTA" and columns start with "PCT_". "PCT_" columns are bounded by 0 and 100. "2PA", "3PA", and "FTA" are bounded because players can only attempt shots in a limited amount of time, so the shot attempts are bounded. 

In [1]:
import numpy as np
import pandas as pd
shot = pd.read_csv('nba_shot_types.csv')
stat = pd.read_csv('Seasons_stats_complete.csv')

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

ohenc = OneHotEncoder(sparse=False)
mmscaler = MinMaxScaler()
sscaler = StandardScaler()

In [6]:
stat1 = stat[stat['Year'] > 2013]
stat.head(5)

shot.head(5)

shot['YEAR'] = shot['YEAR'].replace('2018-2019', 2019)
shot['YEAR'] = shot['YEAR'].replace('2017-2018', 2018)
shot['YEAR'] = shot['YEAR'].replace('2016-2017', 2017)
shot['YEAR'] = shot['YEAR'].replace('2015-2016', 2016)
shot['YEAR'] = shot['YEAR'].replace('2014-2015', 2015)
shot['YEAR'] = shot['YEAR'].replace('2013-2014', 2014)

merged = stat1.merge(shot, left_on='Player', right_on='PLAYER', how='right')
merged1 = merged[merged['Year'] == merged['YEAR']]
merged1 = merged1.reset_index()

mm_columns = ['Age', 'G','MP', 'PER', '2PA', '3PA', 'FTA', 'ORB%', "DRB%",
              'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OBPM', 'DBPM',
              'BPM', 'VORP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
              'ORB', 'TRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
              'W', 'L', "PCT_FGA_2PT", "PCT_FGA_3PT", "PCT_PTS_2PT", 
              "PCT_PTS_MR", "PCT_PTS_3PT", "PCT_PTS_FSTBRK", "PCT_PTS_FT", 
              "PCT_PTS_OFF_TOS", "PCT_PTS_INTHEPT", 'PCT_2PTGFM_ASSTD',
              'PCT_2PTGFM_UNASSTD', 'PCT_3PTGFM_ASSTD', 'PCT_3PTGFM_UNASSTD',
              'PCT_FGM_ASSTD','PCT_FGM_UNASSTD', 'DWS', 'WS']

for name in mm_columns:
    merged1[name] = mmscaler.fit_transform(pd.DataFrame(merged1[name]))

oh_columns = ['TEAM', 'Pos']

one_hot_var = ohenc.fit_transform(pd.DataFrame(merged1[oh_columns]))
one_hot_var_name = ohenc.get_feature_names(oh_columns)
one_hot_var = pd.DataFrame(one_hot_var, columns = [one_hot_var_name])


df = merged1[['Age', 'G','MP', 'PER', '2PA', '3PA', 'FTA', 'ORB%', "DRB%",
             'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OBPM', 'DBPM',
             'BPM', 'VORP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA',
             'ORB', 'TRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
             'W', 'L', "PCT_FGA_2PT", "PCT_FGA_3PT", "PCT_PTS_2PT", 'WS',
             "PCT_PTS_MR", "PCT_PTS_3PT", "PCT_PTS_FSTBRK", "PCT_PTS_FT", 
             "PCT_PTS_OFF_TOS", "PCT_PTS_INTHEPT", 'PCT_2PTGFM_ASSTD',
             'PCT_2PTGFM_UNASSTD', 'PCT_3PTGFM_ASSTD', 'PCT_3PTGFM_UNASSTD',
             'PCT_FGM_ASSTD','PCT_FGM_UNASSTD', 'TEAM', 'Pos', 'OWS', 'DWS',
             'TS%', '3PAr', 'FTr', 'WS/48', 'FG%', '3P%', '2P%', 'eFG%', 'FT%']]

df1 = pd.concat([df, one_hot_var], axis=1)
#df1.to_csv('NBA_preprocessed.csv', index=False)
df1.head(5)

,Age,G,MP,PER,2PA,3PA,FTA,ORB%,DRB%,TRB%,...,"(TEAM_SAC,)","(TEAM_SAS,)","(TEAM_TOR,)","(TEAM_UTA,)","(TEAM_WAS,)","(Pos_C,)","(Pos_PF,)","(Pos_PG,)","(Pos_SF,)","(Pos_SG,)"
0,0.391304,0.109756,0.039052,0.254131,0.002111,0.014591,0.011351,0.027,0.201,0.179936,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.347826,0.841463,0.434699,0.284330,0.051372,0.284047,0.068104,0.031,0.170,0.159236,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.304348,0.451220,0.178297,0.304843,0.056298,0.087549,0.068104,0.039,0.180,0.175159,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.304348,0.060976,0.015045,0.229630,0.007037,0.006809,0.003405,0.046,0.152,0.154459,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.304348,0.378049,0.162932,0.312251,0.049261,0.080739,0.064699,0.038,0.182,0.176752,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
"""EDA"""


pd.read_csv('NBA_preprocessed.csv').head(5)

,Age,G,MP,PER,2PA,3PA,FTA,ORB%,DRB%,TRB%,...,"('TEAM_SAC',)","('TEAM_SAS',)","('TEAM_TOR',)","('TEAM_UTA',)","('TEAM_WAS',)","('Pos_C',)","('Pos_PF',)","('Pos_PG',)","('Pos_SF',)","('Pos_SG',)"
0,0.391304,0.109756,0.039052,0.254131,0.002111,0.014591,0.011351,0.027,0.201,0.179936,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.347826,0.841463,0.434699,0.284330,0.051372,0.284047,0.068104,0.031,0.170,0.159236,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.304348,0.451220,0.178297,0.304843,0.056298,0.087549,0.068104,0.039,0.180,0.175159,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.304348,0.060976,0.015045,0.229630,0.007037,0.006809,0.003405,0.046,0.152,0.154459,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.304348,0.378049,0.162932,0.312251,0.049261,0.080739,0.064699,0.038,0.182,0.176752,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [27]:
"""Old Preprocessing Code"""

# stat = stat[['Year', 'Player', 'Pos', 'Age', 'OWS', "2PA", "3PA", "FTA"]]
# stat1 = stat[stat['Year'] > 2013]
# stat.head(5)

# shot = shot[['YEAR', 'PLAYER', "PCT_FGA_2PT", "PCT_FGA_3PT",
#              "PCT_PTS_2PT", "PCT_PTS_MR", "PCT_PTS_3PT", "PCT_PTS_FSTBRK",
#              "PCT_PTS_FT", "PCT_PTS_OFF_TOS", "PCT_PTS_INTHEPT"]]
# shot.head(5)

# shot['YEAR'] = shot['YEAR'].replace('2018-2019', 2019)
# shot['YEAR'] = shot['YEAR'].replace('2017-2018', 2018)
# shot['YEAR'] = shot['YEAR'].replace('2016-2017', 2017)
# shot['YEAR'] = shot['YEAR'].replace('2015-2016', 2016)
# shot['YEAR'] = shot['YEAR'].replace('2014-2015', 2015)
# shot['YEAR'] = shot['YEAR'].replace('2013-2014', 2014)

# stat['2PA'] = mmscaler.fit_transform(pd.DataFrame(stat['2PA']))
# stat['3PA'] = mmscaler.fit_transform(pd.DataFrame(stat['3PA']))
# stat['FTA'] = mmscaler.fit_transform(pd.DataFrame(stat['FTA']))
# shot['PCT_FGA_2PT'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_FGA_2PT']))
# shot['PCT_FGA_3PT'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_FGA_3PT']))
# shot['PCT_PTS_2PT'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_2PT']))
# shot['PCT_PTS_2PT'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_2PT']))
# shot['PCT_PTS_MR'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_MR']))
# shot['PCT_PTS_3PT'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_3PT']))
# shot['PCT_PTS_FSTBRK'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_FSTBRK']))
# shot['PCT_PTS_FT'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_FT']))
# shot['PCT_PTS_OFF_TOS'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_OFF_TOS']))
# shot['PCT_PTS_INTHEPT'] = mmscaler.fit_transform(pd.DataFrame(shot['PCT_PTS_INTHEPT']))


# merged = stat1.merge(shot, left_on='Player', right_on='PLAYER', how='right')
# merged1 = merged[merged['Year'] == merged['YEAR']]
# merged2 = merged1[['Pos', 'OWS', '2PA', '3PA', 'FTA', "PCT_FGA_2PT", "PCT_FGA_3PT",
#                   "PCT_PTS_2PT", "PCT_PTS_MR", "PCT_PTS_3PT", "PCT_PTS_FSTBRK",
#                   "PCT_PTS_FT", "PCT_PTS_OFF_TOS", "PCT_PTS_INTHEPT"]]
# index = [x for x in range(3553)]

# one_hot_var = ohenc.fit_transform(pd.DataFrame(merged2['Pos']))
# one_hot_var_name = ohenc.get_feature_names(['Pos'])
# one_hot_var = pd.DataFrame(one_hot_var, columns = [one_hot_var_name])

# merged2 = merged2.reset_index()
# df = merged2[['OWS', '2PA', '3PA', 'FTA', "PCT_FGA_2PT", "PCT_FGA_3PT",
#                   "PCT_PTS_2PT", "PCT_PTS_MR", "PCT_PTS_3PT", "PCT_PTS_FSTBRK",
#                   "PCT_PTS_FT", "PCT_PTS_OFF_TOS", "PCT_PTS_INTHEPT"]]

# df1 = pd.concat([df, one_hot_var], axis=1)
# #one_hot_var
# 1df1